### Class 7- Starter code


In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics


### Create sample data and fit a model

In [2]:
df = pd.DataFrame({'x': range(100), 'y': range(100)})
biased_df  = df.copy()
biased_df.loc[:20, 'x'] = 1
biased_df.loc[:20, 'y'] = 1

def append_jitter(series):
    jitter = np.random.random_sample(size=100)
    return series + jitter

df['x'] = append_jitter(df.x)
df['y'] = append_jitter(df.y)

biased_df['x'] = append_jitter(biased_df.x)
biased_df['y'] = append_jitter(biased_df.y)


In [3]:
## fit
lm = linear_model.LinearRegression().fit(df[['x']], df['y'])
print metrics.mean_squared_error(df['y'], lm.predict(df[['x']])), lm.score(df[['x']], df['y'])


0.137434921414 0.999834710486


In [4]:
## biased fit
lm = linear_model.LinearRegression().fit(biased_df[['x']], biased_df['y'])
print metrics.mean_squared_error(df['y'], lm.predict(df[['x']])), lm.score(df[['x']], df['y'])

0.148099603885 0.999821884341


## Cross validation
#### Intro to cross validation with bike share data from last time. We will be modeling casual ridership. 

In [41]:
from sklearn import cross_validation

bikeshare = pd.read_csv('/Users/brianna/GeneralAssembly/data-science/DS-SF-32/dataset/bikeshare.csv')

#### Create dummy variables and set outcome (dependent) variable

In [42]:
weather = pd.get_dummies(bikeshare.weathersit, prefix='weather')
modeldata = bikeshare[['temp', 'hum']].join(weather[['weather_1', 'weather_2', 'weather_3']])
y = bikeshare.casual 

#### Create a cross valiation with 5 folds

In [43]:
kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)

In [44]:
mse_values = []
scores = []
n= 0
print "~~~~ CROSS VALIDATION each fold ~~~~"
for train_index, test_index in kf:
    lm = linear_model.LinearRegression().fit(modeldata.iloc[train_index], y.iloc[train_index])
    mse_values.append(metrics.mean_squared_error(y.iloc[test_index], lm.predict(modeldata.iloc[test_index])))
    scores.append(lm.score(modeldata, y))
    n+=1
    print 'Model', n
    print 'MSE:', mse_values[n-1]
    print 'R2:', scores[n-1]


print "~~~~ SUMMARY OF CROSS VALIDATION ~~~~"
print 'Mean of MSE for all folds:', np.mean(mse_values)
print 'Mean of R2 for all folds:', np.mean(scores)

~~~~ CROSS VALIDATION each fold ~~~~
Model 1
MSE: 1724.4756582
R2: 0.311917364482
Model 2
MSE: 1755.49277399
R2: 0.31185022168
Model 3
MSE: 1573.18271458
R2: 0.31191867186
Model 4
MSE: 1756.86321992
R2: 0.31191993155
Model 5
MSE: 1558.4656105
R2: 0.311810478281
~~~~ SUMMARY OF CROSS VALIDATION ~~~~
Mean of MSE for all folds: 1673.69599544
Mean of R2 for all folds: 0.311883333571


In [45]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print "~~~~ Single Model ~~~~"
print 'MSE of single model:', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'R2: ', lm.score(modeldata, y)

~~~~ Single Model ~~~~
MSE of single model: 1672.58110765
R2:  0.311934605989


### Check
While the cross validated approach here generated more overall error, which of the two approaches would predict new data more accurately: the single model or the cross validated, averaged one? Why?


Answer: 

### There are ways to improve our model with regularization. 
Let's check out the effects on MSE and R2

In [46]:
lm = linear_model.LinearRegression().fit(modeldata, y)
print "~~~ OLS ~~~"
print 'OLS MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'OLS R2:', lm.score(modeldata, y)

lm = linear_model.Lasso().fit(modeldata, y)
print "~~~ Lasso ~~~"
print 'Lasso MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'Lasso R2:', lm.score(modeldata, y)

lm = linear_model.Ridge().fit(modeldata, y)
print "~~~ Ridge ~~~"
print 'Ridge MSE: ', metrics.mean_squared_error(y, lm.predict(modeldata))
print 'Ridge R2:', lm.score(modeldata, y)

~~~ OLS ~~~
OLS MSE:  1672.58110765
OLS R2: 0.311934605989
~~~ Lasso ~~~
Lasso MSE:  1725.41581608
Lasso R2: 0.290199495922
~~~ Ridge ~~~
Ridge MSE:  1672.60490113
Ridge R2: 0.311924817843


### Figuring out the alphas can be done by "hand"

In [59]:
alphas = np.logspace(-10, 10, 11)
for a in alphas:
    print 'Alpha:', a
    lm = linear_model.Ridge(alpha=a)
    lm.fit(modeldata, y)
    print lm.coef_
    print metrics.mean_squared_error(y, lm.predict(modeldata))
    print

Alpha: 1e-10
[ 51.69074648 -85.42093527 -26.19155729 -22.36688219 -22.58646932
  69.18155582]
1669.1855824

Alpha: 1e-08
[ 51.6907465  -85.42093527 -26.19155705 -22.36688196 -22.58646908
  69.1815558 ]
1669.1855824

Alpha: 1e-06
[ 51.69074827 -85.42093505 -26.19153343 -22.36685837 -22.58644552
  69.18155366]
1669.1855824

Alpha: 0.0001
[ 51.69092607 -85.4209127  -26.18917193 -22.36449969 -22.5840898
  69.18133976]
1669.1855824

Alpha: 0.01
[ 51.70867303 -85.41868071 -25.95535923 -22.13096958 -22.35085574
  69.1599898 ]
1669.1855924

Alpha: 1.0
[ 53.22517884 -85.21373538 -14.2571925  -10.45997799 -10.71111107
  67.33307328]
1669.21333685

Alpha: 100.0
[ 58.12987799 -71.87906992  -0.67213486   1.11521991  -0.98919788
  51.91324355]
1677.86187919

Alpha: 10000.0
[ 6.71550275 -5.0663266   3.2403735  -1.18816916 -2.04432385  5.91742393]
2276.40291095

Alpha: 1000000.0
[ 0.07572782 -0.05727313  0.05519188 -0.02735333 -0.02773977  0.06677023]
2428.76750376

Alpha: 100000000.0
[ 0.00075829 -0.

### Or we can use grid search to make this faster

In [60]:
from sklearn import grid_search

alphas = np.logspace(-10, 10, 11)
gs = grid_search.GridSearchCV(
    estimator=linear_model.Ridge(),
    param_grid={'alpha': alphas},
    scoring='neg_mean_squared_error')

gs.fit(modeldata, y)


GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-10,   1.00000e-08,   1.00000e-06,   1.00000e-04,
         1.00000e-02,   1.00000e+00,   1.00000e+02,   1.00000e+04,
         1.00000e+06,   1.00000e+08,   1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

##### Best score 

In [61]:
print gs.best_score_ 

-1809.13912015


##### mean squared error here comes in negative, so let's make it positive.

In [62]:
print -gs.best_score_ 

1809.13912015


##### explains which grid_search setup worked best

In [63]:
print gs.best_estimator_ 

Ridge(alpha=100.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


##### shows all the grid pairings and their performances.

In [64]:
print gs.grid_scores_ 

[mean: -1820.38585, std: 541.19811, params: {'alpha': 1e-10}, mean: -1820.38585, std: 541.19811, params: {'alpha': 1e-08}, mean: -1820.38585, std: 541.19811, params: {'alpha': 9.9999999999999995e-07}, mean: -1820.38571, std: 541.19819, params: {'alpha': 0.0001}, mean: -1820.37226, std: 541.20656, params: {'alpha': 0.01}, mean: -1819.35741, std: 542.08731, params: {'alpha': 1.0}, mean: -1809.13912, std: 630.13589, params: {'alpha': 100.0}, mean: -2427.39792, std: 947.49974, params: {'alpha': 10000.0}, mean: -2541.01259, std: 963.94437, params: {'alpha': 1000000.0}, mean: -2542.41923, std: 964.11282, params: {'alpha': 100000000.0}, mean: -2542.43333, std: 964.11450, params: {'alpha': 10000000000.0}]


In [56]:
modeldata['atemp'] = bikeshare['atemp']

modeldata.head()

,temp,hum,weather_1,weather_2,weather_3,atemp
0,0.24,0.81,1,0,0,0.2879
1,0.22,0.80,1,0,0,0.2727
2,0.22,0.80,1,0,0,0.2727
3,0.24,0.75,1,0,0,0.2879
4,0.24,0.75,1,0,0,0.2879


In [65]:
alphas = np.logspace(-10, 10, 11)
for a in alphas:
    print 'Alpha:', a
    lm = linear_model.Lasso(alpha=a)
    lm.fit(modeldata, y)
    print lm.coef_
    print metrics.mean_squared_error(y, lm.predict(modeldata))
    print

Alpha: 1e-10
[ 51.72716089 -85.40149146 -15.01477699 -11.19514725 -11.41844027
  69.12170681]
1669.20711682

Alpha: 1e-08
[ 51.72716249 -85.40149108 -15.01469077 -11.19506105 -11.41835405
  69.12170458]
1669.20711716

Alpha: 1e-06
[ 51.72732199 -85.40145226 -15.00606846 -11.18644099 -11.40973182
  69.12148185]
1669.20715039

Alpha: 0.0001
[ 51.74327087 -85.39757118 -14.14424466 -10.32484119 -10.54791564
  69.09921124]
1669.21060285

Alpha: 0.01
[ 53.50668003 -85.00838146  -3.50619809   0.20352207  -0.          66.78872146]
1669.28450222

Alpha: 1.0
[ 86.81079432 -55.76414394   0.          -0.          -0.           0.        ]
1725.41581608

Alpha: 100.0
[ 0. -0.  0. -0. -0.  0.]
2430.84614081

Alpha: 10000.0
[ 0. -0.  0. -0. -0.  0.]
2430.84614081

Alpha: 1000000.0
[ 0. -0.  0. -0. -0.  0.]
2430.84614081

Alpha: 100000000.0
[ 0. -0.  0. -0. -0.  0.]
2430.84614081

Alpha: 10000000000.0
[ 0. -0.  0. -0. -0.  0.]
2430.84614081



## Gradient Descent

In [71]:
num_to_approach, start, steps, optimized = 6.2, 0., [-1, 1], False
while not optimized:
    current_distance = num_to_approach - start
    got_better = False
    better = None
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            better = n
            print distance, 'is less distance away than', current_distance
            current_distance = distance
            start = n
    if got_better:
        print 'found better solution! at distance', current_distance
#         print 'a is ', a
        print 'better n is ', better
        # alpha from example above
#         a += 1
    else:
        optimized = True
        print start, 'is closest to', num_to_approach
    print


5.2 is less distance away than 6.2
found better solution! at distance 5.2
better n is  1.0

4.2 is less distance away than 5.2
found better solution! at distance 4.2
better n is  2.0

3.2 is less distance away than 4.2
found better solution! at distance 3.2
better n is  3.0

2.2 is less distance away than 3.2
found better solution! at distance 2.2
better n is  4.0

1.2 is less distance away than 2.2
found better solution! at distance 1.2
better n is  5.0

0.2 is less distance away than 1.2
found better solution! at distance 0.2
better n is  6.0

6.0 is closest to 6.2



### Bonus: 
implement a stopping point, similar to what n_iter would do in gradient descent when we've reached "good enough"

In [74]:
num_to_approach, start, steps, optimized = 6.2, 0., [-1, 1], False
max_num_iter = 4
iter_count = 0
while (not optimized) and (iter_count < max_num_iter):
    iter_count += 1
    current_distance = num_to_approach - start
    got_better = False
    better = None
    next_steps = [start + i for i in steps]
    for n in next_steps:
        distance = np.abs(num_to_approach - n)
        if distance < current_distance:
            got_better = True
            better = n
            print distance, 'is less distance away than', current_distance
            current_distance = distance
            start = n
    if got_better:
        print 'found better solution! at distance', current_distance
#         print 'a is ', a
        print 'better n is ', better
        # alpha from example above
#         a += 1
    else:
        optimized = True
        print start, 'is closest to', num_to_approach
    print


5.2 is less distance away than 6.2
found better solution! at distance 5.2
better n is  1.0

4.2 is less distance away than 5.2
found better solution! at distance 4.2
better n is  2.0

3.2 is less distance away than 4.2
found better solution! at distance 3.2
better n is  3.0

2.2 is less distance away than 3.2
found better solution! at distance 2.2
better n is  4.0



## Demo: Application of Gradient Descent 

In [78]:
lm = linear_model.SGDRegressor()
lm.fit(modeldata, y)
print "Gradient Descent R2:", lm.score(modeldata, y)
print "Gradient Descent MSE:", metrics.mean_squared_error(y, lm.predict(modeldata))

Gradient Descent R2: 0.31003117407
Gradient Descent MSE: 1677.20805779


### Check: Untuned, how well did gradient descent perform compared to OLS?

Answer: 

# Independent Practice: Bike data revisited

There are tons of ways to approach a regression problem. The regularization techniques appended to ordinary least squares optimizes the size of coefficients to best account for error. Gradient Descent also introduces learning rate (how aggressively do we solve the problem), epsilon (at what point do we say the error margin is acceptable), and iterations (when should we stop no matter what?)

For this deliverable, our goals are to:

- implement the gradient descent approach to our bike-share modeling problem,
- show how gradient descent solves and optimizes the solution,
- demonstrate the grid_search module!

While exploring the Gradient Descent regressor object, you'll build a grid search using the stochastic gradient descent estimator for the bike-share data set. Continue with either the model you evaluated last class or the simpler one from today. In particular, be sure to implement the "param_grid" in the grid search to get answers for the following questions:

- With a set of alpha values between 10^-10 and 10^-1, how does the mean squared error change?
- Based on the data, we know when to properly use l1 vs l2 regularization. By using a grid search with l1_ratios between 0 and 1 (increasing every 0.05), does that statement hold true? If not, did gradient descent have enough iterations?
- How do these results change when you alter the learning rate (eta0)?

**Bonus**: Can you see the advantages and disadvantages of using gradient descent after finishing this exercise?

### Starter Code

In [53]:
params = {} # put your gradient descent parameters here
gs = grid_search.GridSearchCV(
    estimator=linear_model.SGDRegressor(),
    cv=cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True),
    param_grid=params,
    scoring='mean_squared_error',
    )

gs.fit(modeldata, y)

print 'BEST ESTIMATOR'
print -gs.best_score_
print gs.best_estimator_
print 'ALL ESTIMATORS'
print gs.grid_scores_

BEST ESTIMATOR
1690.50260151
SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)
ALL ESTIMATORS
[mean: -1690.50260, std: 71.85895, params: {}]


In [19]:
## go for it!